In [1]:
!pip install ipyfilechooser

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import ipywidgets as widgets
from ipyfilechooser import FileChooser
from IPython.display import display, Markdown, clear_output
import yaml
import subprocess
import copy
import os

In [3]:
# Global variables to store the file path and YAML data
HtTomo_base_yaml_path = None
initial_data = None
current_data = None

# Create a FileChooser widget with labels
yaml_default_path = "/dls/k11/data/2025/mg38035-1/xml/tomography/configurations/HTTomo/"

In [4]:
# Add labels
file_chooser = FileChooser(yaml_default_path)
file_chooser.title = 'HtTomo yaml file:'
path_label = widgets.Label(value="HtTomo yaml file:")
# Function to save the file path in the global variable and load initial data
def save_file_path(chooser):
    global HtTomo_base_yaml_path, initial_data, current_data
    if chooser.selected:
        HtTomo_base_yaml_path = chooser.selected
        with open(HtTomo_base_yaml_path, 'r') as file:
            initial_data = yaml.safe_load(file)
            current_data = copy.deepcopy(initial_data)
        print(f"Selected file path: {HtTomo_base_yaml_path}")
        # Display the initial YAML content after loading the file
        #display_yaml_entries()
        reset_all()
# Attach the function to the file selection
file_chooser.register_callback(save_file_path)
# Display the labels and FileChooser widget
#display(path_label, file_chooser)

In [5]:
def update_current_data(path, new_value):
    global current_data
    path_components = path.split('.')
    
    current_level = current_data
    
    for component in path_components[:-1]:
        try:
            index = int(component) - 1
            if isinstance(current_level, list):
                current_level = current_level[index]
            else:
                keys = list(current_level.keys())
                current_level = current_level[keys[index]]
        except (ValueError, TypeError):
            current_level = current_level[component]
    
    last_component = path_components[-1]
    try:
        index = int(last_component) - 1
        if isinstance(current_level, list):
            current_level[index] = new_value
        else:
            keys = list(current_level.keys())
            current_level[keys[index]] = new_value
    except (ValueError, TypeError):
        current_level[last_component] = new_value

In [6]:
def remove_entry(path, button):
    global current_data, display_area, buttons
    path_components = path.split('.')
    
    current_level = current_data
    for component in path_components[:-1]:
        try:
            index = int(component) - 1
            if isinstance(current_level, list):
                current_level = current_level[index]
            else:
                keys = list(current_level.keys())
                current_level = current_level[keys[index]]
        except (ValueError, TypeError):
            current_level = current_level[component]
    
    last_component = path_components[-1]
    try:
        index = int(last_component) - 1
        if isinstance(current_level, list):
            current_level.pop(index)
        else:
            keys = list(current_level.keys())
            del current_level[keys[index]]
    except (ValueError, TypeError):
        del current_level[last_component]
    
    # Remove the button from the display
    buttons.remove(button)
    display_area.children = [child for child in display_area.children if button not in child.children]
    
    # Check if current_data is empty and update the display area
    if not current_data:
        display_area.children = []
    
    display_widgets()

In [7]:
def reset_all():
    global current_data, initial_data, buttons, display_area
    current_data = copy.deepcopy(initial_data)
    display_area.children = []  # Clear the first row's content
    buttons = []  # Reset the buttons list
    display_widgets()  # Redisplay the widgets from scratch

In [8]:
# Create a FileChooser widget for saving the file
save_file_chooser = FileChooser(file_chooser.default_path)
save_file_chooser.title = 'Save To:'
save_file_chooser.show_only_dirs = False

def save_current_data(chooser):
    global current_data
    if chooser.selected:
        save_path = chooser.selected
        #with open(save_path, 'w') as file:
        #    yaml.dump(current_data, file)
        #print(f"Current data saved to {save_path}")
    else:
        print("No file selected for saving.")

def open_save_file_chooser(button):
    global save_file_chooser, HtTomo_base_yaml_path
    if HtTomo_base_yaml_path:
        base_path, ext = os.path.splitext(HtTomo_base_yaml_path)
        default_save_path = f"{base_path}_edit{ext}"
        save_file_chooser.default_filename = os.path.basename(default_save_path)
    display(save_file_chooser)

In [9]:
def overwrite_current_data(button):
    global current_data, save_file_chooser
    if save_file_chooser.selected:
        save_path = save_file_chooser.selected
        with open(save_path, 'w') as file:
            yaml.dump(current_data, file)
        print(f"Current data overwritten at {save_path}")
    else:
        print("No save path selected. Overwrite action aborted.")

In [10]:
# Function to check if a value is a terminal (end-tree) entry
def is_terminal(value):
    return not isinstance(value, (dict, list))

def calculate_max_label_width(entry, indent=0):
    max_length = 0
    for key, value in entry.items():
        number_space = "-" if indent == 0 else ". " * indent
        display_key = f"{number_space} {key}"
        max_length = max(max_length, len(display_key))
        if isinstance(value, dict):
            max_length = max(max_length, calculate_max_label_width(value, indent + 1))
        elif isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    max_length = max(max_length, calculate_max_label_width(item, indent + 1))
    return max_length

def display_widgets():
    global current_data, buttons, display_area, reset_button, save_button
    buttons = []
    widgets_list = []
    
    dropdown_layout = widgets.Layout(width='400px')  # Set a fixed width for the dropdown menu
    remove_button_width = '100px'  # Set a fixed width for the remove button
    
    if isinstance(current_data, list):
        max_label_width = max(calculate_max_label_width(entry) for entry in current_data)
        for entry_index, entry in enumerate(current_data, 1):
            entry_widgets = create_widgets(entry, prefix=f"{entry_index}.", max_label_width=max_label_width)
            
            remove_button = widgets.Button(
                description='Remove',
                layout=widgets.Layout(width=remove_button_width, margin='5px 0px 5px 0px')
            )
            remove_button.on_click(lambda b, path=f"{entry_index}", button=remove_button: remove_entry(path, button))
            buttons.append(remove_button)
            
            dropdown = widgets.Accordion(children=[widgets.VBox(entry_widgets)], layout=dropdown_layout)
            
            # Use the value of the first key as the title
            first_key = list(entry.keys())[0]
            title = str(entry[first_key])
            dropdown.set_title(0, title)
            
            widgets_list.append(
                widgets.HBox([remove_button, dropdown], layout=widgets.Layout(margin='0px 0px 10px 0px'))
            )
    elif isinstance(current_data, dict):
        max_label_width = calculate_max_label_width(current_data)
        entry_widgets = create_widgets(current_data, max_label_width=max_label_width)
        remove_button = widgets.Button(
            description='Remove',
            layout=widgets.Layout(width=remove_button_width, margin='5px 0px 5px 0px')
        )
        remove_button.on_click(lambda b, path="Configuration", button=remove_button: remove_entry(path, button))
        buttons.append(remove_button)
        
        dropdown = widgets.Accordion(children=[widgets.VBox(entry_widgets)], layout=dropdown_layout)
        dropdown.set_title(0, "Configuration")
        
        widgets_list.append(
            widgets.HBox([remove_button, dropdown], layout=widgets.Layout(margin='0px 0px 10px 0px'))
        )
    #else:
    #    print("Select YAML structure")
    
    display_area.children = widgets_list  # Update the first row's content

    # Set the width of the reset and save buttons to match the remove button width
    reset_button.layout.width = remove_button_width
    save_button.layout.width = remove_button_width

def create_widgets(entry, indent=0, prefix="", max_label_width=0):
    widgets_list = []
    label_layout = widgets.Layout(min_width=f'{max_label_width + 10}px', margin='0px 5px 0px 0px')  # Set a minimum width and right margin for labels
    widget_layout = widgets.Layout(width='auto')  # Flexible width for entry widgets
    
    for i, (key, value) in enumerate(entry.items(), 1):
        number_space = "-" if indent == 0 else ". " * indent
        display_key = f"{number_space} {key}"
        widget_path = f"{prefix}{key}"
        
        if is_terminal(value):
            if isinstance(value, bool):
                entry_widget = widgets.Dropdown(
                    options=[True, False],
                    value=value,
                    description='',
                    layout=widget_layout
                )
                entry_widget.observe(lambda change, path=widget_path: update_current_data(path, change['new']), names='value')
            elif isinstance(value, (int, float)):
                entry_widget = widgets.Text(
                    value=str(value), 
                    description='', 
                    layout=widget_layout
                )
                entry_widget.observe(
                    lambda change, path=widget_path, orig_type=type(value): 
                    update_current_data(path, orig_type(change['new'])), 
                    names='value'
                )
            elif isinstance(value, str) and '/' in value:
                entry_widget = widgets.Text(
                    value=value, 
                    description='', 
                    layout=widget_layout
                )
                entry_widget.observe(lambda change, path=widget_path: update_current_data(path, change['new']), names='value')
            else:
                entry_widget = widgets.Label(value=str(value), layout=widget_layout)
            
            widgets_list.append(
                widgets.HBox([
                    widgets.Label(value=display_key + ":", layout=label_layout),
                    entry_widget
                ], layout=widgets.Layout(margin='0px 0px 0px 0px'))
            )
        else:
            widgets_list.append(widgets.Label(value=display_key + ":", layout=label_layout))
            if isinstance(value, dict):
                sub_widgets = create_widgets(value, indent + 1, f"{prefix}{key}.", max_label_width)
                widgets_list.extend(sub_widgets)
            elif isinstance(value, list):
                for j, item in enumerate(value, 1):
                    if isinstance(item, dict):
                        sub_widgets = create_widgets(item, indent + 1, f"{prefix}{key}.{j}.", max_label_width)
                        widgets_list.extend(sub_widgets)
                    else:
                        widgets_list.append(
                            widgets.HBox([
                                widgets.Label(value=f"{number_space} - {item}", layout=label_layout)
                            ])
                        )
    return widgets_list

In [11]:
# Create the display area and reset/save buttons
display_area = widgets.VBox()  # Create the display area for the first row

reset_button = widgets.Button(
    description='Reset All',
    layout=widgets.Layout(width='100px', margin='10px 0px 0px 0px')  # Align to the left
)
reset_button.on_click(lambda b: reset_all())

save_button = widgets.Button(
    description='Save',
    layout=widgets.Layout(width='100px', margin='10px 0px 0px 10px')  # Align to the left with some margin
)
save_button.on_click(lambda b: save_current_data())

# Attach the save function to the file chooser
save_file_chooser.register_callback(save_current_data)

# Create the Overwrite button
overwrite_button = widgets.Button(
    description='Save',
    layout=widgets.Layout(width='100px', margin='10px 0px 0px 0px')
)
overwrite_button.on_click(overwrite_current_data)

# Update the save button to open the file chooser
save_button.on_click(open_save_file_chooser)

# Create a horizontal line
horizontal_line = widgets.HTML(value="<hr style='width: 500px; margin-left: 0;'>")

# Combine the display area, file chooser, and reset/save buttons into a figure with two rows
buttons_box = widgets.HBox([reset_button])
figure = widgets.VBox([file_chooser, display_area, buttons_box, horizontal_line, save_file_chooser, overwrite_button])
display(figure)

# Display the updated YAML entries with remove buttons
display_widgets()

In [12]:
print(current_data)

None


In [13]:
print(initial_data)

None
